In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Path to the model directory
model_dir = '../LLMs/Qwen3-30B-A3B'

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForCausalLM.from_pretrained(model_dir)

# Use CPU
device = torch.device("cpu")
model.to(device)


/home/zheng/miniconda3/envs/qwen3-cpu/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 16/16 [02:26<00:00,  9.13s/it]


Qwen3MoeForCausalLM(
  (model): Qwen3MoeModel(
    (embed_tokens): Embedding(151936, 2048)
    (layers): ModuleList(
      (0-47): 48 x Qwen3MoeDecoderLayer(
        (self_attn): Qwen3MoeAttention(
          (q_proj): Linear(in_features=2048, out_features=4096, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=4096, out_features=2048, bias=False)
          (q_norm): Qwen3MoeRMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3MoeRMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MoeSparseMoeBlock(
          (gate): Linear(in_features=2048, out_features=128, bias=False)
          (experts): ModuleList(
            (0-127): 128 x Qwen3MoeMLP(
              (gate_proj): Linear(in_features=2048, out_features=768, bias=False)
              (up_proj): Linear(in_features=2048, out_features=768, bias=False)
              (down_proj): Linear

In [3]:
# Prepare the model input
prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True # Switch between thinking and non-thinking modes. Default is True.
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# Initialize the input ids for the generation
input_ids = model_inputs['input_ids']

# Generate tokens one by one
output_ids = input_ids
previous_output = []

# Loop to generate and print tokens one by one
for _ in range(50):  # Max 50 new tokens
    # Generate the next token
    output = model.generate(
        input_ids=output_ids,
        max_new_tokens=1,  # Generate only 1 token at a time
        do_sample=True,  # Enable sampling for randomness
        pad_token_id=tokenizer.eos_token_id,  # Ensure no padding
        return_dict_in_generate=True,  # Get the dict output for token-level information
        output_scores=True  # Get the scores for each token, though we don't need them
    )
    
    # Get the new token generated
    new_token_id = output.sequences[0, -1].item()
    new_token = tokenizer.decode([new_token_id], skip_special_tokens=True)
    
    # Only print new tokens and avoid repeating lines
    if new_token != previous_output[-1:]:
        print(new_token, end="", flush=True)
    
    # Append current token to previous output for comparison
    previous_output.append(new_token)
    
    # Update the input_ids for the next token
    output_ids = output.sequences

# After loop, print the final part of content
print()  # To end the line cleanly


<think>
Okay, the user wants a short introduction to large language models. Let me start by defining what they are. I should mention that they're AI systems trained on vast text data. Maybe include examples like GPT or BERT.

Next,
